In [36]:
import json
import pandas as pd
import os
import shutil
from datetime import datetime

In [3]:
DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

In [24]:
store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

menu_columns = (
    "id", # 메뉴 고유번호
    "store", # 음식점 고유번호
    "menu_name", # 메뉴 이름
    "price", # 메뉴 가격
)

user_columns = (
    "id", # 유저 고유번호
    "gender", # 유저 성별
    "age", # 유저 나이
)

In [10]:
with open(DATA_FILE, encoding="UTF-8") as f:
    data = json.loads(f.read())

[{'id': 101,
  'name': '1004임자도횟집',
  'branch': None,
  'area': '괴안동',
  'tel': '010-9990-8188',
  'address': '경기도 부천시 괴안동 108-6',
  'latitude': '37.482738',
  'longitude': '126.811292',
  'category_list': [],
  'menu_list': [{'menu': '광어(소)', 'price': 22000},
   {'menu': '광어(중)', 'price': 25000},
   {'menu': '광어(대)', 'price': 45000},
   {'menu': '우럭(중)', 'price': 25000},
   {'menu': '광어 + 우럭(중)', 'price': 35000},
   {'menu': '광어 + 우럭(대)', 'price': 45000}],
  'bhour_list': [{'type': 1,
    'week_type': 1,
    'mon': 1,
    'tue': 1,
    'wed': 1,
    'thu': 1,
    'fri': 1,
    'sat': 1,
    'sun': 1,
    'start_time': '12:00:00',
    'end_time': '24:00:00',
    'etc': ''}],
  'review_cnt': 0,
  'review_list': []},
 {'id': 102,
  'name': '1004카페',
  'branch': '',
  'area': '서귀포',
  'tel': '010-4849-7653',
  'address': '제주특별자치도 서귀포시 대정읍 구억리 1004',
  'latitude': '33.275735',
  'longitude': '126.279001',
  'category_list': [{'category': '더치베이비'}, {'category': '브런치'}],
  'menu_list': [],
 

In [39]:
stores = [] # 음식점 테이블
reviews = [] # 리뷰 테이블
menus = [] # 메뉴 테이블
users = [] # 유저 테이블
users_id_list = []

for d in data:
    # 음식점 분류의 값을 추출하여, 리스트화
    categories = [c["category"] for c in d["category_list"]]
    # store_columns의 순서에 맞도록 값을 넣는다.
    stores.append([
        d["id"],
        d["name"],
        d["branch"],
        d["area"],
        d["tel"],
        d["address"],
        d["latitude"],
        d["longitude"],
        "|".join(categories),
    ])
    
    # 메뉴 데이터 추출
    for menu in d["menu_list"]:
        menus.append([
            len(menus) + 1,
            d["id"],
            menu["menu"],
            menu["price"],
        ])
    
    # 리뷰 및 유저 데이터 추출
    for review in d["review_list"]:
        review_info = review["review_info"]
        user_info = review["writer_info"]
        
        # 리뷰 데이터 추출
        reviews.append([
            review_info["id"],
            d["id"],
            user_info["id"],
            review_info["score"],
            review_info["content"],
            review_info["reg_time"]
        ])
        
        # 유저 데이터 추출
        # 유저 아이디가 중복되지 않도록 하려면?
        if user_info["id"] not in users_id_list:
            users.append([
                user_info["id"],
                user_info["gender"],
                datetime.now().year - int(user_info["born_year"]) + 1,
            ])
            users_id_list.append(user_info["id"])

In [23]:
stores[0]

[1,
 'Agal',
 None,
 '홍대',
 '010-6689-5886',
 '서울특별시 마포구 동교동 170-13',
 '37.556862',
 '126.926666',
 '아구찜|포장마차']

In [22]:
reviews[0]

[1,
 15,
 68632,
 5,
 '전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!',
 '1970-01-01 00:00:00']

In [26]:
menus[0]

[1, 3, '메인 (피쉬앤칩스, 오리 가슴살 등)', 14000]

In [40]:
users[0:10]

[[68632, '남', 32],
 [389728, '여', 29],
 [68716, '여', 38],
 [774353, '여', 50],
 [115682, '남', 31],
 [388883, '여', 54],
 [180541, '남', 53],
 [249051, '여', 29],
 [52424, '남', 33],
 [946187, '여', 29]]

In [42]:
store_frame = pd.DataFrame(data=stores, columns=store_columns)

In [43]:
store_frame.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트


In [45]:
review_frame = pd.DataFrame(data=reviews, columns=review_columns)
review_frame.head()

,id,store,user,score,content,reg_time
0,1,15,68632,5,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,1970-01-01 00:00:00
1,2,18,389728,5,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,1970-01-01 00:00:00
2,3,19,68716,4,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,1970-01-01 00:00:00
3,4,37,774353,2,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",1970-01-01 00:00:00
4,5,38,115682,3,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2019-03-15 22:16:47


In [46]:
menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
menu_frame.head()

,id,store,menu_name,price
0,1,3,"메인 (피쉬앤칩스, 오리 가슴살 등)",14000.0
1,2,3,스타터 보드(파테 등),18000.0
2,3,3,디저트 (초콜렛 퐁당 등),8500.0
3,4,3,생과일 칵테일,9000.0
4,5,3,생맥주,7000.0


In [47]:
user_frame = pd.DataFrame(data=users, columns=user_columns)
user_frame.head()

,id,gender,age
0,68632,남,32
1,389728,여,29
2,68716,여,38
3,774353,여,50
4,115682,남,31


In [49]:
data = pd.read_pickle(DUMP_FILE)

In [53]:
shutil.get_terminal_size()[0] - 1

119